【AI 2019选修课 大作业】  
这个题目是我一直想探索的，借着人工智能大作业的选题把它提出来，也刚好也算是大创项目的一个子任务。     
数据集来源于一位学长推荐的[课程](https://www.bilibili.com/video/av32279263?p=1)， 也借鉴了里面的部分思路。  
最初的创新点本来是想结合歌词主题建模+协同过滤，但是在粤语歌主题建模中发现效果极差，一方面都是粤语歌相似性过高（这个经验恰好也用在了之后的主题建模任务中），另一方面，歌曲由于语言的差异等，反而不是很适合主题建模。但是我又想用主题建模，所以就利用了数据集里面的所有文本信息，然后发现其实效果还不错，于是就变成了歌单描述主题建模+协同过滤的整个思路。  
然后比较麻烦的一点是没法用surprise库，看遍StackOverflow也没找到解决办法，所以也是借此机会自己写了一遍协同过滤（但程序的效率说明我太菜了..）。
写着写着就大概理解了，这两者都是在运用集体智慧，还挺有意思，不过也还有很多没解决的事情。但对于大作业已经基本完整了。

----


#  1 数据预处理

## 1.1 筛选需要的数据项

首先对15G的歌单数据进行处理，选取了数据集中歌单的信息，包括歌单名、歌单标签、歌单id、歌单描述、歌单收藏数、歌曲信息，每一个歌单下包含着多首歌曲，每一首歌曲的信息包括歌曲id、歌曲名称、歌手、歌曲热度。 （15G的数据量较大，在jupyter跑不起来，python文件可以）  
最终输出一个playlist_info的txt文件，经过收藏量的筛选，只剩下24951个歌单。

In [ ]:
import os
import tarfile
import pickle as pk
import sys
import json

def readPlaylist(in_line):
    data = json.loads(in_line)
    name = data['result']['name'] #歌单名称
    tags = ",".join(data['result']['tags']) # 标签以逗号分隔的字符串存储
    subscribed_count = data['result']['subscribedCount'] # 收藏数
    try:
        description=data['result']['description'] #歌单的描述
    except Exception as e:
        description=" "
    if(subscribed_count<100): # 筛选小于100收藏的歌单
        return False
    playlist_id = data['result']['id'] # 歌单的id名
    song_info = ''
    songs = data['result']['tracks'] # 歌曲
    for song in songs: 
        try:
            song_info += "\t"+":::".join([str(song['id']),song['name'],song['artists'][0]['name'],str(song['popularity'])])
            song_info=song_info.replace("\n"," ")
            # 歌曲信息：歌曲id:::歌曲名称:::歌手:::歌曲热度
        except Exception as e:
            continue
     # 歌单信息： 歌单名##歌单标签##歌单id##歌单描述##歌单收藏数##歌曲信息（以:::分割的信息）
    final_info=str(name+"##"+tags+"##"+str(playlist_id)+"##"+str(description)+"##"+str(subscribed_count)+song_info)
    return final_info.replace("\n"," ")
   
def readFile(in_file, out_file):
    out = open(out_file, 'w',encoding="utf-8")
    num=0
    for line in open(in_file,encoding='utf-8'):
        result = readPlaylist(line)
        #print(result)
        num=num+1
        print("------------------",num,"--------------------")
        if(result):
            out.write(str(result)+"\n")  
    out.close()
    
if __name__=="__main__":
    readFile("./playlistdetail.all.json", "./playlist_info_utf8.txt")

## 1.2 构建歌曲（歌单）id和名称的字典

在这一步构建了歌曲和歌单的两个属性id和名称对应的字典。

In [ ]:
import pickle as pk
import sys

def getDict(i, dic_playlist, dic_song):
    contents = i.strip().split("\t")
    playlist = contents[0].split("##")
    # 歌单信息： 歌单名##歌单标签##歌单id##歌单描述##歌单收藏数##歌曲信息（以:::分割的信息）
    dic_playlist[playlist[2]] = playlist[0]
    for i in contents[1:]:
        try:
            song_info = i.split(":::")
            dic_song[song_info[0]] = song_info[1]+"\t"+song_info[2]
        except:
            print("error"+i+"\n")


def readFile(in_file, out_playlist, out_song):
    dic_playlist = {}  # dic_playlist[歌单id]=歌单名称
    dic_song = {}  # dic_song[歌曲id]=歌曲名称
    num = 0
    for i in open(in_file, encoding="utf-8"):
        num = num+1
        print("------------------", num, "--------------------")
        getDict(i, dic_playlist, dic_song)
    pk.dump(dic_playlist, open(out_playlist, "wb"))
    pk.dump(dic_song, open(out_song, "wb"))


if __name__ == "__main__":
    readFile("./playlist_info_utf8.txt", "playlist.pkl", "song.pkl")

## 1.3 歌单文本信息分词（utf-8存储）
为了后续的LDA主题模型，对歌单的文本信息进行分词。
歌单的文本信息包括歌单名称、歌单标签和歌单描述。歌单的文本信息中除了中英文，还会有很多表情符（使用unicode编码），虽然表情符一定程度上可以表征歌单的情感以及歌曲的情感，当然要具体细究那就是后续的工作了，此处需要将这些表情符进行过滤。  
1.将需要使用的文本信息进行提取，提取后存储在utf8_playlist.txt中。

In [ ]:
import os
import tarfile
import pickle as pk
import sys
import json

def readPlaylist(in_line):
    data = json.loads(in_line)
    name = data['result']['name'] #歌单名称
    tags = ",".join(data['result']['tags']) # 标签以逗号分隔的字符串存储
    try:
        description=data['result']['description'] #歌单的描述
    except Exception as e:
        description="---"
    description=(str(description)).replace("\n","  ")
    subscribed_count = data['result']['subscribedCount'] # 收藏数
    if(subscribed_count<100):
        return False
    playlist_id = data['result']['id'] # 歌单的id名
    return name+"##"+tags+"##"+str(playlist_id)+"##"+str(description)
    # 歌单信息： 歌单名##歌单标签##歌单id##歌单描述

def readFile(in_file, out_file):
    out=open(out_file, 'w',encoding="utf-8")
    num=0
    for line in open(in_file,encoding='utf-8'):
        result=readPlaylist(line)
        num=num+1
        print("------------------",num,"--------------------")
        # print(result)
        if(result):
            out.write(str(result)+"\n")  
    out.close()
    
if __name__=="__main__":
    readFile("D:\playlistdetail.all.json", "./utf8_playlist.txt")

2.将所有的文本信息（标签、名称、描述）合在一起，由此汇成一个“文档”，这一个文档可认为是对歌单的总体描述，然后使用jieba分词，每一个文档分词的结果用list存储起来，最后歌单id和分词后的list以字典形式存储在dic_splited_sentence.pkl。  
（这里也可以考虑加入歌单里的每一首歌的名称，甚至歌的歌词进行主题建模，但由于收集歌词的难度较大，就没有做到这一步）  
由于歌单的描述中会混合非中文的其他语言文字或者一些颜文字、表情等，为了简便分词，将这些词都过滤，并且也根据导入的停用词表，过滤停用词，在这里会根据后面LDA模型的生成情况，添加一些针对歌单描述的特定停用词，如“歌曲”、“专辑”、“歌手”、“年度”、“单曲”、“歌单”等。

In [ ]:
import json
import sys
import chardet
import jieba
import pickle as pk
import re
import emoji
import pandas as pd

def getSplitedSentence(i, dic_splited_sentence, stopwords):
    # 获取歌单信息: 歌单名##歌单标签##歌单id##歌单描述
    contents = i.strip().split("\t")
    playlist = contents[0].split("##")
    sentence = " ".join([playlist[0], playlist[1], playlist[3]])
    chinese_sentence = re.sub('[^\u4e00-\u9fa5]', '', sentence)  # 过滤非中文
    word_list = jieba.lcut(chinese_sentence)
    word_list = filter(lambda x: len(x) > 1, word_list)
    word_list = filter(lambda x: x not in stopwords, word_list)  # 过滤停用词
    word_list = list(word_list)
    # print(word_list)
    dic_splited_sentence[playlist[2]] = word_list


def readFile(in_file, out_pk, stopwords):
    dic_splited_sentence = {}
    num = 0
    for i in open(in_file, encoding="utf-8"):
        print("------------", num, "-------------")
        num = num+1
        getSplitedSentence(i, dic_splited_sentence, stopwords)
    pk.dump(dic_splited_sentence, open(out_pk, "wb"))


if __name__ == "__main__":
    stopwords = pd.read_csv("./stopwords.txt", index_col=False,
                            quoting=3, sep="\t", names=['stopword'], encoding='utf-8')
    stopwords = stopwords['stopword'].values
    readFile("./playlist_info_utf8.txt",
             "./dic_splited_sentence.pkl", stopwords)

# 2 歌单文本信息LDA主题建模
LDA主题建模的思想是认为一篇文章的产生是服从概率分布的，每个词以一定概率选择了某个主题，每一个主题以一定概率选择了一篇文章。简单地说，LDA是一种主题抽取模型，通过无监督学习从文档中提出主题以及主题中的词语。最终LDA主题建模能够判断文档之间的相似性。因此可以考虑将歌单相关的文本信息都作为一个文档，通过LDA主题建模来进行歌单的聚类。  
以下使用gensim库中的函数，进行LDA主题建模，实现了对歌单的聚类，此处将词袋模型和lda模型的相关参数分别保存在dictionary.pkl和playlist_lda.model中。  

In [ ]:
from gensim import corpora, models, similarities
import gensim
import pickle as pkl

all_sentence = []
dic = pkl.load(open("dic_splited_sentence.pkl", "rb"))
for (key, value) in dic.items():
    all_sentence.append(value)
# 词袋模型
dictionary = corpora.Dictionary(all_sentence)
print(type(dictionary))
pkl.dump(dictionary, open("./model/dictionary.pkl", "wb"))

corpus = [dictionary.doc2bow(i) for i in all_sentence]
print(corpus[5])
# lda训练
lda = gensim.models.ldamodel.LdaModel(
    corpus=corpus, id2word=dictionary, num_topics=30)
for topic in lda.print_topics(num_topics=30, num_words=5):
    print(topic[1])

lda.save('./model/playlist_lda.model')

LDA主题模型将所有的文本分为30个主题（类别数可自己确定，此处设置30个，解释性较好），每一个主题下有很多词，下图中每一行是一个主题，包括该主题里的词语以及出现概率。
![lda.png](./lda.png)
可以看到LDA主题建模的效果较好，是具有解释性的。

# 3 协同过滤推荐 
为了实现推荐，考虑使用基于用户的协同过滤方法。这一方法最初是使用用户对不同电影的评分来计算相似度，并找到与某一用户最相似的前k个用户，最终推荐的结果是该用户未看过但是与他最相似的用户看过的电影。    
这一过程可类比到歌曲推荐：假设一个用户收藏了某一歌单， 那么在针对该用户的歌曲推荐中可以结合该收藏歌单来进行推荐。具体来说，LDA主题建模已经将歌单分为了30个类别，每一个歌单所属类别概率最高的那一类的编号（1-30）就可以作为该歌单的“类别评分”，歌单里的歌曲相当于继承了歌单的这个“类别评分”，以此代替原来协同过滤方法的用户对电影的主观评分（毕竟用户只是进行了收藏歌单行为，并没有主观地给歌曲打分，但是我们可以认为用户进行收藏行为，就是部分认可了这个歌单对歌曲的类别判断）。根据这个歌单评分，我们就能够用协同过滤算法找到与该歌单最相似的k个歌单，然后返回最相似的歌单中原歌单未出现的歌曲进行推荐。 
推荐系统相关的算法在surprise库中有较为完整的实现，可以调用surprise库中的相关算法进行实现。

## 3.1 构建符合suprise的数据类型
读取模型和构建好的dictionary 根据这一模型，能够获得歌单最符合的主题。

In [ ]:
lda = models.ldamodel.LdaModel.load('./model/playlist_lda.model')
for topic in lda.print_topics(num_topics=30, num_words=5):
    print(topic[1])

dic = pkl.load(open("dic_splited_sentence.pkl","rb"))
doc_list=dic['486555874']
print(doc_list)
dictionary=pkl.load(open("./model/dictionary.pkl","rb"))
bow = [dictionary.doc2bow(doc_list)]
print(bow)
print(list(lda.get_document_topics(bow)))

这里就将歌曲解析成【歌单id 歌曲id 评分】的格式（surprise库处理的格式），并按照该格式 格式存入formatted_music.txt中。  

In [ ]:
#coding: utf-8
import json
import sys
from gensim import corpora, models, similarities
import pickle as pkl

lda = models.ldamodel.LdaModel.load('./model/playlist_lda.model')  # 加载模型
dic = pkl.load(open("dic_splited_sentence.pkl", "rb"))  # 读入字典
dictionary = pkl.load(open("./model/dictionary.pkl", "rb"))


def isNull(s):
    return len(s.split(",")) > 2


def getSongId(song_info):
    # 歌曲信息：歌曲id:::歌曲名称:::歌手:::歌曲热度
    try:
        song = song_info.split(":::")
        return song[0]
    except Exception as e:
        return ""


def getSongScore(x):
    doc_list = dic[x]
    bow = [dictionary.doc2bow(doc_list)]
    distribution = list(lda.get_document_topics(bow))
    score = 0
    maxp = 0
    for i in distribution[0]:
        if(maxp < i[1]):
            score, maxp = i[0], i[1]
    # print(score,maxp)
    return score+1


def formatInfo(in_line):
    try:
        contents = in_line.strip().split("\t")
        playlist = contents[0].split("##")
        # 歌单信息playlist contents[0]： 歌单名##歌单标签##歌单id##歌单描述##歌单收藏数
        # 歌曲信息content[1:]（每一条是以:::分割的信息）
        songScore = getSongScore(playlist[2])
        songs_info = map(
            lambda x: playlist[2]+",,"+str(getSongId(x))+",,"+str(songScore), contents[1:])
        songs_info = filter(isNull, songs_info)
        return "\n".join(songs_info)
    except Exception as e:
        print(e)
        return False


def readFile(in_file, out_file):
    out = open(out_file, 'w', encoding="utf-8")
    num = 0
    for line in open(in_file, encoding="utf-8"):
        print("-----", num, "-------")
        num = num+1
        result = formatInfo(line)
        if(result):
            out.write(result.strip()+"\n")
    out.close()


readFile("./playlist_info_utf8.txt", "./formatted_music.txt")


##  3.2 使用surprise库完成推荐
这里可以使用surprise库的协同过滤算法，并通过KNN算法求出与某一个歌单最相近的topK个歌单。

In [ ]:
import os
import io
import pickle as pkl
import pandas as pd
import numpy as np
from surprise import dataset
from surprise import KNNBaseline, Reader

id2name =pkl.load(open("playlist.pkl","rb"))
name2id = {}
for id in id2name:
    name2id[id2name[id]] = id

file_path = os.path.expanduser('./formatted_music.txt')
reader = Reader(line_format='user item rating', sep=',')
music = dataset.load_from_file(file_path, reader=reader)
trainset = music.build_full_trainset() # 数据集准备
algo = KNNBaseline() #KNN算法
algo.train(trainset)

test = name2id.keys()[39] 
test_id = name2id[test]
test_inner_id = algo.trainset.to_inner_uid(test_id)
print("歌单名称:", test,"歌单id:", test_id,"内部id:", test_inner_id)
playlist_neighbors = algo.get_neighbors(test_inner_id, k=10)

playlist_neighbors = (algo.trainset.to_raw_uid(inner_id)
                       for inner_id in playlist_neighbors)
playlist_neighbors = (id2name[playlist_id]
                       for playlist_id in playlist_neighbors)

print(test, "最接近的10个歌单为：")
for i in playlist_neighbors:
    print(i, algo.trainset.to_inner_uid(name2id[i]))


## 3.3 实现协同过滤算法
本来使用已经写好的surprise是非常方便的，但是surprise尽管能安装，但是无法导入，因此我决定直接自己写协同过滤的算法。  
首先，使用formatted_music.txt，使用getData函数，将一些不符合常规的行去除；接着建立一个二维字典，并存入listsong.pkl，字典的键是歌单的原id，字典的值是一个字典，该字典的键是歌曲的原id,值是歌曲类别评分（一个歌单下的歌曲类别评分相同）。本来最开始打算建立的是一个歌单与歌曲的array，但是因为有二十多万首歌，有内存限制。  
接着使用余弦相似度进行计算，通过getSimilarityMatrix函数获得一个大小为playlist_num（24949）维的矩阵，并存入sim_matrix.npy中。（计算相似度矩阵就花了两小时多一点，算了三亿多次相似度，过后再想想怎么优化算法）

In [ ]:
import os
import io
import pickle as pkl
import pandas as pd
import numpy as np
import re
import time
from math import *


def getInnerId(dic, key):
    # 输入:字典key 输出:key对应的内部id
    num = 0
    for (k, v) in dic.items():
        if(k != key):
            num = num+1
        else:
            return num


def getDataset(in_file):
    # 输出二维数组 每一列是 歌单id 歌曲id 类别评分
    dataset = []
    for line in open(in_file, encoding="utf-8"):
        line = line.strip().split(',,')  # 单逗号无法分割，古典曲目题目会有逗号
        if(line[1].isdigit()):  # 保证歌单id都是数字，解决取不到歌曲id的问题（非法字符会出现歌曲id是歌曲名的情况）
            dataset.append(line)
    dataset = np.array(dataset)
    return dataset


def addInnerDic(dic, key_a, key_b, val):
    # 二维字典辅助函数
    if key_a in dic:
        dic[key_a].update({key_b: val})
    else:
        dic.update({key_a: {key_b: val}})


def getListSongDic(dataset, out_pkl):
        # 输入：dataset 格式[歌单id,歌曲id,类别评分]
        # 输出：out_pkl 保存了一个二维字典 {歌单：{歌曲id1：类别评分(string)，歌曲id2:类别评分....}}
    dataset_dic = {}
    for i in dataset:  # 2998219
        playlist_id, songlist_id, item_score = i[0], i[1], i[2]+".0"
        addInnerDic(dataset_dic, playlist_id, songlist_id, item_score)
    pkl.dump(dataset_dic, open(out_pkl, "wb"))


def getIdDic(in_pkl):
    # 输入：pkl文件 key:value 输出：out_dic字典 内部id:原id
    in_dic = pkl.load(open(in_pkl, "rb"))
    out_dic = {}
    id = 0
    for (k, v) in in_dic.items():
        out_dic[id] = k
        id = id+1
    return out_dic


def getDistance(pid, list2song_dic):
    # 输入：歌单i 歌单歌曲字典
    # 输出：歌单i向量长度
    # i为内部id
    di = 0
    if (pid not in list2song_dic):
        return 1.0
    for(k, v) in list2song_dic[pid].items():
        di = di+float(v)**2
    return sqrt(di)


def getSimilarity(pid_i, pid_j, dis_i, dis_j, list2song_dic):
    # 输入：歌单i（内部id） 歌单j（内部id） 歌单i向量长度 歌单j向量长度 歌单歌曲字典
    # 输出：歌单i和歌单j的相似度
    numerator = 0
    for(k, v) in list2song_dic[pid_i].items():
        if(k in list2song_dic[pid_j].keys()):
            numerator = numerator+float(v)*float(list2song_dic[pid_j][k])
    return numerator/(dis_i*dis_j)


def getSimilarityMatrix(playlist_pkl, listsong_pkl):
    # 输入：歌单字典【歌单id：歌单名】 歌单歌曲字典【歌单id:{歌曲id1:歌曲名1，歌单id2:歌曲名2...}...】
    # 输出：相似度矩阵sim_matrix sim_matrix[i][j]存储原id为i和j的歌单的相似度
    playlist = pkl.load(open(playlist_pkl, "rb"))
    playlist_num = len(playlist)  # 歌单数 24949
    id2pid_dic = getIdDic(playlist_pkl)
    list2song_dic = pkl.load(open(listsong_pkl, "rb"))
    sim_matrix = np.zeros((playlist_num, playlist_num),
                          dtype=np.float)  # 相似度矩阵
    dis_list = []  # 存储每一个歌单的向量长度
    for i in range(0, playlist_num):
        di = getDistance(id2pid_dic[i], list2song_dic)
        dis_list.append(di)
    for i in range(0, playlist_num):
        print(i)
        if (id2pid_dic[i] in list2song_dic):
            for j in range(i, playlist_num):
                if(id2pid_dic[j] in list2song_dic):
                    sim_matrix[i][j] = getSimilarity(
                        id2pid_dic[i], id2pid_dic[j], dis_list[i], dis_list[j], list2song_dic)
    np.save("sim_matrix.npy", sim_matrix)
    return sim_matrix


def topk(i, sim_matrix, k):
    # 输入第i个歌单id 输出最接近的list的前k个歌单
    sim_list = []
    return sim_list


dataset = getDataset("./formatted_music.txt")
getListSongDic(dataset, "./listsong.pkl")
getSimilarityMatrix("./playlist.pkl", "./listsong.pkl")

## 3.4 获取推荐结果
使用getTopK函数，获取与歌单最相近的k个歌单，并返回最相似的歌单未出现在原歌单中的歌曲信息。

In [ ]:
import pickle as pkl
import numpy as np


def getIdDic(in_pkl):
    # 输入：pkl文件 key:value 输出：out_dic字典 内部id:原id
    in_dic = pkl.load(open(in_pkl, "rb"))
    out_dic = {}
    id = 0
    for (k, v) in in_dic.items():
        out_dic[id] = k
        id = id+1
    return out_dic


def getTopK(pid, sim_matrix, k, playlist_pkl):
    # 输入：歌单内部id 相似度矩阵 取相似的歌单数k 歌单字典 歌单原id:歌单名
    # 输出：打印前k个最相似的歌单原id、歌单名和相似度 并返回最相似的歌单
    playlist = pkl.load(open(playlist_pkl, "rb"))
    id2pid_dic = getIdDic(playlist_pkl)
    result_dic = {}
    for i in range(0, pid):
        sim_matrix[pid][i] = sim_matrix[i][pid]  # 补齐矩阵
    result_list = sim_matrix[pid].tolist()
    for i in range(0, len(result_list)):
        result_dic[id2pid_dic[i]] = result_list[i]  # result_dic 歌单原id:相似度
    sorted_result = []
    sorted_sim_value = []
    for i in sorted(result_dic, key=result_dic.__getitem__, reverse=True):
        sorted_result.append(i)
        sorted_sim_value.append(result_dic[i])
    print("与", id2pid_dic[pid], ":", playlist[id2pid_dic[pid]], " 最相似的歌单为:")
    for i in range(1, k+1):
        print(sorted_result[i], ":", playlist[sorted_result[i]],
              " 相似度为:", sorted_sim_value[i])  # 歌曲id：歌曲名 相似度
    return sorted_result[1]  # 除了本身的最接近的歌单


def getRecommendSong(top1, pid, playlist_pkl, song_pkl, listsong_pkl):
    # 打印最相似歌单中未出现在原歌单的歌曲id 歌曲名 创作者
    print("推荐以下歌曲：")
    song = pkl.load(open(song_pkl, "rb"))
    id2pid_dic = getIdDic(playlist_pkl)
    list2song_dic = pkl.load(open(listsong_pkl, "rb"))
    for (k, v) in list2song_dic[top1].items():
        if (k not in list2song_dic[id2pid_dic[pid]]):
            print(k, ":", song[k])


sim_matrix = np.load("sim_matrix.npy")
for i in range(0, 5):
    top1 = getTopK(i, sim_matrix, 10, "./playlist.pkl")
    getRecommendSong(top1, i, "./playlist.pkl", "./song.pkl", "./listsong.pkl")
    print("---------------------------------------------------")


# 4 结果与小结
## 4.1 推荐结果

![playlist_sim.png](./playlist_sim.png)

---

![recommend.png](./recommend.png)
通过上面的两张图，还是能直观地感觉这样的推荐是靠谱的，但是在衡量准确性的时候会产生一个问题：实际的评分是一个连续值，能够通过算法求出一个估计值，如果有测试数据集就可以计算出实际的评分和估计值的均方根误差等来衡量。但是此处使用的类别评分本质上是一个离散值，难以进行估计值的求解，况且也没有测试集，因此这里就没有进行准确性的衡量。  
但是从协同过滤的整体思想来看，引入LDA主题建模计算出来的类别评分，也是在语言的层面上借助了人类的集体智慧，理论上在实际使用中应该会让人“感觉”推荐得更准。（也不一定，其实还是应该想想怎么衡量准确性）

## 4.2 协同过滤无法解决的问题
1.数据的稀疏性。衡量歌单的相似性时会发现，数据集中一共有二十多万首歌，但一般一个歌单的数量一般不会超过1000首，这就会造成歌单里的歌曲重叠性较低，因此从结果也能够看出，即使是最接近的歌单，其和原歌单的相似度也较低。  
2.另外，当数据量较大时，协同过滤需要花费一定的时间，同时扩展性也更差，当新增了用户或者歌曲时，需要重新计算，难以扩展。  
## 4.3 未来的工作

1.考虑如何衡量引入了LDA主题建模的协同过滤的准确率。  
2.学习和实现冷启动问题的解决方案。  
3.学习和思考算法的效率优化。
4.使用word2vec类比到song2vec的推荐。